# 02 — Preprocessing (Clean & Harmonize)

**Purpose:** Load the country-only panel, fix types and column hygiene, add log transforms for skewed variables, and (optionally) standardize **feature** columns. No target imputation or time-window filtering is done here.

**Inputs:** `../data/processed/countries_only.csv`

**Outputs (CSV only):** `../data/processed/countries_preprocessed.csv`

**Key decisions/assumptions:**
- Countries only; aggregates remain excluded (as in 01).
- No target imputation; `cereal_yield` may remain missing (handled in modeling).
- Do not scale/transform `year`.
- Create `log_*` companions for selected skewed variables.
- If scaling is enabled, standardize **features only** (exclude `year`, `cereal_yield`, and identifiers).

## 1. Setup
Define stable paths and display options. A local fallback (`/mnt/data`) supports hosted or containerized runs. This cell establishes a consistent entry point across environments.


In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

pd.set_option("display.max_columns", None)

# Consistent with 01 (use ../)
DATA_PATH = Path("../data/processed/countries_only.csv")
FALLBACK  = Path("/mnt/data/countries_only.csv")  # optional
OUT_PATH  = Path("../data/processed/countries_preprocessed.csv")

if not DATA_PATH.exists() and FALLBACK.exists():
    DATA_PATH = FALLBACK
print("Using data from:", DATA_PATH)

Using data from: ../data/processed/countries_only.csv


## 2. Load data
Read the country-only panel produced in 01. The initial shape and a short preview help confirm that the upstream split executed as expected and that columns are present for downstream steps.


In [2]:
df = pd.read_csv(DATA_PATH)
print("Shape:", df.shape)
df.head()

Shape: (14040, 14)


,Country Name,Country Code,year,fertilizer_use,arable_land_pct,precipitation,cereal_yield,co2_total_mt,co2_per_capita,gdp_per_capita,population,rural_pop_pct,temp_anomaly,is_aggregate
0,Afghanistan,AFG,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9035043.0,91.599,-0.03,False
1,Afghanistan,AFG,1961,0.143791,11.728991,327.0,1115.1,NaN,NaN,NaN,9214083.0,91.316,0.06,False
2,Afghanistan,AFG,1962,0.142857,11.805651,327.0,1079.0,NaN,NaN,NaN,9404406.0,91.024,0.03,False
3,Afghanistan,AFG,1963,0.141935,11.882311,327.0,985.8,NaN,NaN,NaN,9604487.0,90.724,0.05,False
4,Afghanistan,AFG,1964,0.141026,11.958972,327.0,1082.8,NaN,NaN,NaN,9814318.0,90.414,-0.20,False


## 3. Data types and ordering
Cast `year` to a nullable integer (`Int64`) to preserve missing years without coercing to floats. Force identifiers (`Country Name`, `Country Code`) to string to avoid unintended numeric interpretation. Sort by `(Country Name, year)` to make joins, comparisons, and diffs deterministic.

In [3]:
# year as nullable integer
if "year" in df.columns:
    df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")

# identifiers as strings
for col in ["Country Code", "Country Name"]:
    if col in df.columns:
        df[col] = df[col].astype(str)

# sort
order_cols = [c for c in ["Country Name", "year"] if c in df.columns]
if order_cols:
    df = df.sort_values(order_cols).reset_index(drop=True)

df.dtypes.head()

Country Name        object
Country Code        object
year                 Int64
fertilizer_use     float64
arable_land_pct    float64
dtype: object

## 4. Log transforms
Create `log_*` companions for right-skewed variables (e.g., `cereal_yield`, `fertilizer_use`, `gdp_per_capita`) using `log1p`. Negative inputs are clipped to zero to prevent invalid logs. Original columns are retained to keep both raw and transformed representations available.

In [4]:
LOG_VARS = [c for c in ["cereal_yield", "fertilizer_use", "gdp_per_capita"] if c in df.columns]

for col in LOG_VARS:
    vals = pd.to_numeric(df[col], errors="coerce").clip(lower=0)
    df[f"log_{col}"] = np.log1p(vals)

print("Created:", [f"log_{c}" for c in LOG_VARS])

Created: ['log_cereal_yield', 'log_fertilizer_use', 'log_gdp_per_capita']


## 5. Optional standardization
Leave scaling disabled by default. When enabled, standardize only numeric **feature** columns and exclude `year`, the **target** (`cereal_yield`), and identifiers. Standardization improves feature comparability for scale-sensitive models (e.g., linear, distance-based), while tree-based models typically do not require it.

In [5]:
ENABLE_SCALING = False  # set True to scale numeric features

if ENABLE_SCALING:
    numeric_cols = df.select_dtypes(include=[np.number, "Float64", "Int64"]).columns.tolist()
    exclude = {"year", "cereal_yield"} | set([c for c in ["Country Code", "Country Name"] if c in df.columns])
    feature_cols = [c for c in numeric_cols if c not in exclude]
    scaler = StandardScaler()
    df.loc[:, feature_cols] = scaler.fit_transform(df[feature_cols])
    print("Scaled columns:", feature_cols)
else:
    print("Scaling disabled.")

Scaling disabled.


## 6. Save output
Write a single artifact: `../data/processed/countries_preprocessed.csv`. This file serves as the canonical input to feature engineering in 03 and keeps the pipeline’s I/O contract simple and auditable.

In [6]:
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(OUT_PATH, index=False)
print("Saved:", OUT_PATH)

Saved: ../data/processed/countries_preprocessed.csv


## 7. Sanity checks
Report final shape, list of columns, and top missing-value rates. These quick summaries surface schema drift and unexpected sparsity before feature engineering.

In [7]:
print("Final shape:", df.shape)
print("Columns (first 20):", list(df.columns)[:20], "...")
df.isna().mean().sort_values(ascending=False).head(10)

Final shape: (14040, 17)
Columns (first 20): ['Country Name', 'Country Code', 'year', 'fertilizer_use', 'arable_land_pct', 'precipitation', 'cereal_yield', 'co2_total_mt', 'co2_per_capita', 'gdp_per_capita', 'population', 'rural_pop_pct', 'temp_anomaly', 'is_aggregate', 'log_cereal_yield', 'log_fertilizer_use', 'log_gdp_per_capita'] ...


log_fertilizer_use    0.300427
fertilizer_use        0.300427
log_cereal_yield      0.281695
cereal_yield          0.281695
precipitation         0.268661
co2_per_capita        0.223077
co2_total_mt          0.223077
gdp_per_capita        0.178490
log_gdp_per_capita    0.178490
arable_land_pct       0.167308
dtype: float64

## 8. Environment
Print key package versions (Python, NumPy, pandas, scikit-learn). Capturing versions supports reproducibility and simplifies debugging across machines.

In [ ]:
import sys
import platform
import numpy
import pandas
import sklearn

print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())
print("NumPy:", numpy.__version__)
print("Pandas:", pandas.__version__)
print("scikit-learn:", sklearn.__version__)

Python: 3.12.11
Platform: macOS-15.6.1-arm64-arm-64bit
NumPy: 2.3.3
Pandas: 2.3.3
scikit-learn: 1.7.2
